In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df=sns.load_dataset("penguins")

In [5]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [6]:
df.shape

(344, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [8]:
df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


In [10]:
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [11]:
null_col=["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g"]
for i in null_col:
    df[i].fillna(df[i].mean(),inplace=True)

In [13]:
df.sex.value_counts()

Male      168
Female    165
Name: sex, dtype: int64

In [14]:
df.sex.fillna("Male",inplace=True)

In [15]:
df.flipper_length_mm=df.flipper_length_mm.astype(int)
df.body_mass_g=df.body_mass_g.astype(int)

In [17]:
from sklearn.preprocessing import LabelEncoder

In [23]:
le=LabelEncoder()
df.sex=le.fit_transform(df.sex)
df.species=le.fit_transform(df.species)
df.island=le.fit_transform(df.island)

In [24]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.10000,18.70000,181,3750,1
1,0,2,39.50000,17.40000,186,3800,0
2,0,2,40.30000,18.00000,195,3250,0
3,0,2,43.92193,17.15117,200,4201,1
4,0,2,36.70000,19.30000,193,3450,0


In [25]:
from sklearn.preprocessing import MinMaxScaler
mm=MinMaxScaler()

In [28]:
x=df.iloc[:,1:-1]
y=df.species

In [31]:
scaled=mm.fit_transform(x)
df_scaled=pd.DataFrame(scaled,columns=x.columns)

In [33]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_scaled,y,test_size=0.2)

In [34]:
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.svm import SVC  #classisfier
clf=SVC()

In [36]:
param=[{'C': [1, 10, 100, 1000], 
               'kernel': ['sigmoid','rbf','poly'], 
               'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search=GridSearchCV(clf,param_grid=param,scoring="accuracy",n_jobs=-1,cv=5)

In [37]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                    0.9],
                          'kernel': ['sigmoid', 'rbf', 'poly']}],
             scoring='accuracy')

In [38]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'C': 1000, 'gamma': 0.2, 'kernel': 'rbf'}
0.9963636363636365


In [39]:
clf=SVC(C= 1000, gamma= 0.2, kernel= 'rbf')
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)

In [40]:
from sklearn.metrics import accuracy_score,classification_report

In [42]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        29
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00        24

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

